In [2]:
import torch
import torch.nn as nn
import numpy as np
from collections import Counter

class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(TextGenerator, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        device = next(self.parameters()).device
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device))

def prepare_data(text, seq_length=10):  # Уменьшил длину последовательности
    words = text.split()
    word_counts = Counter(words)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    word_to_idx = {word: i for i, word in enumerate(vocab)}
    idx_to_word = {i: word for i, word in enumerate(vocab)}

    sequences = []
    for i in range(seq_length, len(words)):
        seq = words[i-seq_length:i+1]
        sequences.append([word_to_idx[word] for word in seq])

    return np.array(sequences), word_to_idx, idx_to_word, len(vocab)

def train_model(model, sequences, vocab_size, epochs=100, lr=0.01):  # Увеличил learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Переводим модель в режим обучения
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        hidden = model.init_hidden(1)  # Инициализируем hidden для каждой эпохи

        for sequence in sequences:
            optimizer.zero_grad()

            # Подготовка данных
            input_seq = torch.tensor(sequence[:-1]).unsqueeze(0)  # [1, seq_len-1]
            target = torch.tensor(sequence[1:])  # [seq_len-1]

            # Forward pass
            output, hidden = model(input_seq, hidden)

            # Правильный расчет loss
            output = output.squeeze(0)  # [seq_len-1, vocab_size]
            loss = criterion(output, target)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Градиентный clipping
            optimizer.step()

            total_loss += loss.item()

            # Детach и clone hidden states
            hidden = (hidden[0].detach().clone(),
                     hidden[1].detach().clone())

        avg_loss = total_loss / len(sequences)
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')

            # Проверка генерации во время обучения
            if avg_loss < 5.0:  # Только когда модель чему-то научилась
                test_text = generate_text(model, "машинное обучение", word_to_idx, idx_to_word, 5)
                print(f"Тест генерации: {test_text}")

def generate_text(model, start_text, word_to_idx, idx_to_word, num_words=10, temperature=1.0):
    model.eval()
    words = start_text.split()

    if len(words) == 0:
        return ""

    # Проверяем, что все слова есть в словаре
    words = [word for word in words if word in word_to_idx]
    if len(words) == 0:
        return ""

    hidden = model.init_hidden(1)
    generated_text = words.copy()

    # Инициализация hidden state
    with torch.no_grad():
        for word in words[:-1]:
            input_word = torch.tensor([[word_to_idx[word]]])
            _, hidden = model(input_word, hidden)

        current_word = words[-1]

        for _ in range(num_words):
            input_word = torch.tensor([[word_to_idx[current_word]]])
            output, hidden = model(input_word, hidden)

            # Получаем вероятности
            output = output.squeeze(0).squeeze(0)
            probabilities = torch.softmax(output / temperature, dim=0)

            # Выбираем следующее слово
            next_word_idx = torch.multinomial(probabilities, 1).item()
            current_word = idx_to_word.get(next_word_idx, "<UNK>")

            generated_text.append(current_word)

    return ' '.join(generated_text)

# Тестовый текст
text = """машинное обучение это интересная область искусственного интеллекта
которая позволяет компьютерам обучаться на данных и делать прогнозы
нейронные сети являются мощным инструментом для решения сложных задач
глубокое обучение использует многослойные нейронные сети
для распознавания образов и обработки естественного языка"""

print("Подготовка данных...")
sequences, word_to_idx, idx_to_word, vocab_size = prepare_data(text, seq_length=5)
print(f"Размер словаря: {vocab_size}")
print(f"Количество последовательностей: {len(sequences)}")

# Создаем модель
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextGenerator(vocab_size, 64, 128).to(device)  # Уменьшил размерности

print("Начало обучения...")
train_model(model, sequences, vocab_size, epochs=100, lr=0.01)

# Финальная генерация
print("\nФинальная генерация:")
result = generate_text(model, "машинное обучение", word_to_idx, idx_to_word, 15)
print(result)

Подготовка данных...
Размер словаря: 33
Количество последовательностей: 33
Начало обучения...
Epoch 0, Loss: 3.0903
Тест генерации: машинное обучение сети для сети распознавания использует
Epoch 10, Loss: 0.0020
Тест генерации: машинное обучение использует многослойные нейронные сети для
Epoch 20, Loss: 0.0007
Тест генерации: машинное обучение это интересная область искусственного интеллекта
Epoch 30, Loss: 0.0004
Тест генерации: машинное обучение это интересная область искусственного интеллекта
Epoch 40, Loss: 0.0002
Тест генерации: машинное обучение это интересная область искусственного интеллекта
Epoch 50, Loss: 0.0002
Тест генерации: машинное обучение это интересная область искусственного интеллекта
Epoch 60, Loss: 0.0001
Тест генерации: машинное обучение это интересная область искусственного интеллекта
Epoch 70, Loss: 0.0001
Тест генерации: машинное обучение это интересная область искусственного интеллекта
Epoch 80, Loss: 0.0001
Тест генерации: машинное обучение это интересная обл